# Spectral Filtering: Image Resampling

In [ ]:
%matplotlib inline

import numpy as np

import matplotlib.image as img
import matplotlib.pyplot as plt

from scipy.fft import fft2, ifft2, fftshift
from scipy.interpolate import interpn

from skimage import io, exposure
from skimage.filters import gaussian
from skimage.util import img_as_float32 as img_as_float

In [ ]:
def show_image(I, title=None):
    dpi = 80.0
    h, w = I.shape[0]/dpi, I.shape[1]/dpi
    fig, ax = plt.subplots(1, 1, figsize=(w,h))        

    ax.imshow(I, cmap='gray')
    if title != None:
        ax.set_title(title)

In [ ]:
def nextpow2(N):
    n = 1
    while (n<N):
        n *= 2
    return n

In [ ]:
I1 = io.imread("../../images/cars.jpg", as_gray=True)
I1 = img_as_float(I1)

In [ ]:
M, N = I1.shape
M2 = nextpow2(M)
N2 = nextpow2(N)

## Downsampling: Anti-Aliasing + Decimation

In [ ]:
I0 = I1
I0 = gaussian(I0, sigma=2.0)
I0 = I0[0:M:2,0:N:2]

show_image(I0)

## Upsampling: FFT Zero-Padding + Scaling

In [ ]:
I0 = I1
#I0 = gaussian(I1, sigma=0.7071)

F1 = fft2(I0,(M2,N2))

ZM, ZN = (M2//2, N2//2)

F2 = fftshift(F1)
F2 = np.pad(F2, ((ZM,ZM),(ZN,ZN)))
F2 = fftshift(F2)

F2 *= np.prod(np.divide(F2.shape, F1.shape))

I2 = np.real(ifft2(F2))
I2 = I2[:2*M,:2*N]
I2 = np.clip(I2, 0, 1)

show_image(I1)
show_image(I2)

## Upsampling:  Arbitrary Scalar

Method: upsample, apply anti-aliasing filter, downsample. Example below shows $\uparrow 3$ implemented as $\uparrow 4$ followed by $\downarrow 4/3$.

In [ ]:
# SU is upsampling factor
# S1 is nextpower2(SU): SU <= S1
# S2 is rational number: SU = S1/S2
SU = 3
S1 = 4
S2 = 4/3

# Compute fft2(I1) size (M2,N2)
# M2 is nextpower2(M)
# N2 is nextpower2(N)
F1 = fft2(I1,(M2,N2))

# Pad to upsize: (X X/2) X (X/2 X)
# Resulting spectrum: (S1*M2, S1*N2)
ZM, ZN = np.int32((((S1-1)/2)*M2, ((S1-1)/2)*N2))

F3 = fftshift(F1)
F3 = np.pad(F3, ((ZM,ZM), (ZN,ZN)))
F3 = fftshift(F3)

# Compensate for amplitude reduction
F3 *= np.prod(np.divide(F3.shape, F1.shape))

# Extract S1 upsampled image I3
M3, N3 = (S1*M, S1*N)

I3 = np.real(ifft2(F3))
I3 = I3[:M3,:N3]
I3 = np.clip(I3, 0, 1)
         
# Apply anti-aliasing filter
I3 = gaussian(I3, sigma=2.0*S2)

# Size of S2 downsampled image I4
# NOTE: S1*X/S2 = SU*X
M4, N4 = (SU*M, SU*N)

# Extract S2 downsampled image I4
u3, v3 =            (np.linspace(0,M3-1,M3), np.linspace(0,N3-1,N3))
u4, v4 = np.meshgrid(np.linspace(0,M3-1,M4), np.linspace(0,N3-1,N4), indexing='ij')

I4 = interpn((u3,v3), I3, (u4,v4), method='linear')

print('Original image: ', I1.shape)
print('Upsamling x4:   ', I3.shape)
print('Downsampling x3:', I4.shape)

show_image(I4)